
## Whisper 모델 기반 금융 도메인 음성인식
1. 목표

  Whisper 모델을 파인튜닝하여 금융 도메인에 특화된 음성인식 모델을 생성

2.  Dataset 개요
- URL : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=71557
- 이름 : 뉴스 대본 및 앵커 음성 데이터
- 언론에 보도된 뉴스기사, 각 분야(정치, 경제, 사회, 문화, 국제, 지역, 스포츠, IT과학)별 전직,현직 아나운서, 아나운서 교육생들이 뉴스를 보도하는 음성 데이터 1,132시간

### [라이브러리 세부 설명]

- `from transformers import WhisperProcessor, WhisperForConditionalGeneration`
: Hugging Face에서 제공하는 라이브러리. Whisper와 같은 사전 학습된 모델을 로드하고 사용하기 위해 필요. WhisperProcessor는 오디오와 텍스트 데이터를 모델이 이해할 수 있는 형태로 변환하고, WhisperForConditionalGeneration은 실제 음성 인식 모델.

- `from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer`
: Hugging Face Transformers 라이브러리에서 모델을 학습시키는 데 필요한 도구. Seq2SeqTrainingArguments는 학습 설정을 정의하고, Seq2SeqTrainer는 실제 학습 과정을 관리.

- `import evaluate`
: Hugging Face에서 제공하는 라이브러리. 모델의 성능을 평가하는 다양한 지표(Metrics)를 제공. 여기서는 음성 인식의 성능을 측정하는 WER(단어 오류율)을 계산하기 위해 사용.

- `import json`
: JSON 형식의 데이터를 파싱하고 다룰 때 사용되는 파이썬 표준 라이브러리. 데이터셋의 메타데이터가 JSON 파일에 저장되어 있기 때문에 필요.

### [데이터셋 구조 파악]
#### [데이터셋의 전체 디렉토리 구조]
```
data_root_path/
├── Training/
│   ├── 01_SourceData/TS/
│   │   ├── file_a.wav
│   │   ├── ...
│   └── 02_LabeledData/TL/
│       ├── file_b.json
│       ├── ...
│
└── Validation/
    ├── 01_SourceData/VS/
    │   ├── file_c.wav
    │   ├── ...
    └── 02_LabeledData/VL/
        ├── file_d.json
        ├── ...
```

#### [레이블 구조 (JSON)]
- 레이블 데이터는 JSON구조로 되어있다. 
아래처럼 스크립트 정보와 스피커정보, 그리고 오디오파일의 정보를 포함하고 있다.

```
{
	"script": {
		"id": "YTNEC057",
		"url": "http://www.ytn.co.kr/_ln/0102_201801091444153396",
		"title": "최종구 `코스닥 활성화 위해 상장요건 완화·펀드 조성`",
		"press": "YTN",
		"press_field": "경제",
		"press_date": "20180109",
		"index": 2,
		"text": "최 위원장은 오늘 코스닥 시장 활성화 현장간담회에서 이 같은 내용을 담은 코스닥 활성화 방안을 공개했습니다.",
		"sentence_type": "작문형",
		"keyword": "코스닥,상장 요건,코스닥 상장,코스닥 활성화,위원장,코스닥 기업,최종구 코스닥,최종구 코스닥 활성화,활성화,코스닥 시장 활성화"
	},
	"speaker": {
		"id": "SPK054",
		"age": "20대",
		"sex": "남성",
		"job": "아나운서준비생"
	},
	"file_information": {
		"audio_format": "44100 Hz 16bit PCM",
		"utterance_start": "0.445",
		"utterance_end": "7.467",
		"audio_duration": "7.929"
	}
}
```

#### [오디오 데이터 (WAV)]
오디오 데이터는 JSON의 speaker필드 내에 있는 id값과 script필드 내의 id값의 조합 이름의 디렉토리에 .wav파일로 저장되어있다.

예를 들어, 위 JSON 기준으로 `data['speaker']['id']`는 `SPK054`이고, `data['script']['id']`는 `YTNEC057`이다. 

그리고, `speaker`의 `sex`에 따라 데이터파일명의 `F` 또는 `M`이 결정되고, `script`의 `index`에 따라 성별 뒤 나오는 세자리수가 결정된다. 

최종적으로 위 JSON의 파일명은 `SPK054YTNEC057M002.json`이 되고, 이에 매칭되는 WAV파일의 이름은 `SPK054YTNEC057M002.wav`이 된다.

이 규칙에 따라 JSON파일에서 필요한 정보를 파싱하고 해당 정보를 통해 매칭되는 오디오파일을 찾을 수 있다.


예시) 

`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M001.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M002.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M003.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M004.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M005.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M006.json`
`...\data\open_data\Validation\02_LabeledData\VL\SPK054\SPK054YTNEC057\SPK054YTNEC057M007.json`

### 데이터 전처리
- 프로젝트의 목표는 위스퍼 모델을 경제/금융 도메인에 특화되도록 파인튜닝하여 성능을 높이는 것이므로, 데이터셋에서 경제 도메인만 필터링하는 과정이 필요하다. 

- 레이블링 데이터인 JSON파일에 도메인 정보를 포함하고 있어, 해당 부분을 파싱하여 경제 도메인이 내용인 경우에만 해당 데이터를 사용하도록 전처리를 해준다. 

In [ ]:
# 필요한 라이브러리 임포트
import torch
import pandas as pd
from datasets import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import json
import os
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import librosa
import soundfile as sf

warnings.filterwarnings("ignore")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 장치: {device}")

def process_financial_news_data(base_data_path: str, data_type: str):
    """
    레이블링된 뉴스 오디오 데이터를 로드하고, 도메인이 경제인 데이터만 필터링하여 데이터셋으로 반환해주는 함수 

    Args:
        base_data_path (str): 데이터셋의 베이스 경로
        data_type (str): 데이터 유형 ('Training', 'Validation')
    """
    audio_paths = []
    transcriptions = []

    # TS, TL, VS, VL 디렉토리 경로를 설정하기
    if data_type == "Training":
        audio_data_base_path = os.path.join(base_data_path, data_type, "01_SourceData", "TS")
        json_data_path = os.path.join(base_data_path, data_type, "02_LabeledData", "TL")
        print(f"훈련 데이터 오디오 경로: {audio_data_base_path}")
        print(f"훈련 데이터 JSON 경로: {json_data_path}")
    elif data_type == "Validation":
        audio_data_base_path = os.path.join(base_data_path, data_type, "01_SourceData", "VS")
        json_data_path = os.path.join(base_data_path, data_type, "02_LabeledData", "VL")
        print(f"검증 데이터 오디오 경로: {audio_data_base_path}")
        print(f"검증 데이터 JSON 경로: {json_data_path}")
    
    if not os.path.exists(json_data_path):
        print(f"경로를 찾을 수 없습니다: {json_data_path}")
        return None

    # JSON 파일을 순회
    for root, _, files in os.walk(json_data_path):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(root, file)
                
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    
                    # 경제 도메인만 필터링
                    if data['script']['press_field'] == '경제':
                        # JSON 파일에서 필요한 데이터만 파싱
                        speaker_id = data['speaker']['id']
                        script_id = data['script']['id']
                        transcription = data['script']['text']
                        sex = data['speaker']['sex']
                        index = data['script']['index']

                        sex_char = 'M' if sex == '남성' else 'F'
                        index_padded = f"{index:03d}"

                        audio_file_name = f"{speaker_id}{script_id}{sex_char}{index_padded}.wav"
                        
                        audio_path = os.path.join(audio_data_base_path, speaker_id, f"{speaker_id}{script_id}", audio_file_name)

                        # 오디오 파일이 실제로 존재하는지 확인. 존재하면 해당 경로와 스크립트를 리스트에 append
                        if os.path.exists(audio_path):
                            audio_paths.append(audio_path)
                            transcriptions.append(transcription)
                        else:
                            print(f"Warning: {data_type} 데이터에서 파일을 찾을 수 없습니다. 경로: {audio_path}")
                
                except KeyError as e:
                    print(f"Error: {data_type} 데이터의 JSON 파일 {json_path}에 필요한 키가 누락되었습니다: {e}")
                except Exception as e:
                    print(f"Error processing {json_path}: {e}")
                    
    # 오디오 파일 경로와 스크립트를 데이터프레임 형태로 변환
    df = pd.DataFrame({"path": audio_paths, "transcription": transcriptions})

    # 오디오 파일 경로와 스크립트가 저장된 데이터프레임을 Hugging Face의 Dataset 객체로 변환
    dataset = Dataset.from_pandas(df)
    
    print(f"총 {len(dataset)}개의 {data_type}용 경제 도메인 데이터를 찾았습니다.")
    print(df.head())
    
    return dataset

def compute_metrics(pred, processor, metric):
    """모델 성능(WER) 계산 함수"""
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """데이터 콜레이터: 배치 단위로 데이터 패딩 처리"""
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels
        return batch

# prepare_dataset
def prepare_dataset(batch, processor):
    audio_file_path = batch["path"]
    
    try:
        # sf.read를 사용해 오디오 데이터를 직접 읽기
        audio, sampling_rate = sf.read(audio_file_path)
        
        # Whisper 모델에 맞게 sampling_rate 재설정
        if sampling_rate != 16000:
            audio = librosa.resample(audio, orig_sr=sampling_rate, target_sr=16000)
            sampling_rate = 16000
            
        # 특징 벡터 추출 및 토큰화
        batch["input_features"] = processor.feature_extractor(audio, sampling_rate=sampling_rate).input_features[0]
        batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
        return batch

    except Exception as e:
        print(f"오디오 파일 로딩 오류: {audio_file_path} - {e}")
        return {"input_features": None, "labels": None}

def main():
    """전체 모델 훈련 및 평가 워크플로우를 실행하는 메인 함수"""
    
    # 데이터셋 최상위 경로 설정
    base_data_path = "C:/Users/Admin/Desktop/github/voice-recognition-using-whisper/data/news_scripts_and_speech_data/open_data"

    # 훈련 및 평가 여부를 설정하세요
    run_training = False
    run_evaluation = True

    # 1. 데이터셋 로드 (조건부 로드)
    train_dataset, eval_dataset = None, None

    if run_training:
        print("--- 훈련 데이터 로드 시작 ---")
        train_dataset = process_financial_news_data(base_data_path, "Training")
        if train_dataset is None:
            print("훈련 데이터 로드 실패. 프로그램을 종료합니다.")
            return

    if run_evaluation:
        print("\n--- 검증 데이터 로드 시작 ---")
        eval_dataset = process_financial_news_data(base_data_path, "Validation")
        if eval_dataset is None:
            print("검증 데이터 로드 실패. 프로그램을 종료합니다.")
            return
            
    # 2. 모델, 프로세서 로드 (훈련 또는 평가가 필요한 경우에만)
    if not (run_training or run_evaluation):
        print("훈련 또는 평가 작업이 설정되지 않았습니다. 종료합니다.")
        return

    model_name = "openai/whisper-small"
    processor = WhisperProcessor.from_pretrained(model_name)
    model = WhisperForConditionalGeneration.from_pretrained(model_name)
    model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="ko", task="transcribe")
    model.config.suppress_tokens = []
    
    # 3. 데이터 전처리 (모델 로드 후)
    if run_training:
        print("\n훈련 데이터 전처리 중입니다...")
        # map은 Hugging Face의 datasets 라이브러리에서 제공하는 매우 중요한 기능
        # 데이터셋의 모든 개별 항목(행)에 대해 특정 함수를 효율적으로 적용하는 개념
        # 코드에서 map의 역할
        # train_dataset.map(...)은 train_dataset의 모든 행에 대해 prepare_dataset이라는 함수를 적용하라는 의미
        train_dataset = train_dataset.map(
            prepare_dataset, # 모든 데이터에 적용할 함수
            remove_columns=train_dataset.column_names, # 변환 후 제거할 컬럼 설정
            num_proc=os.cpu_count(), # 병렬 처리 설정 (시스템의 CPU 코어 수 사용)
            fn_kwargs={"processor": processor} # prepare_dataset 함수에 전달할 추가 인자
        ).filter(
            lambda x: x["input_features"] is not None
        )
        print(f"필터링 후 훈련 데이터셋 크기: {len(train_dataset)}")
        print("훈련 데이터 전처리 완료.")
    
    # main 함수 내에서 num_proc 변경
    if run_evaluation:
        print("\n검증 데이터 전처리 중입니다...")
        eval_dataset = eval_dataset.map(
            prepare_dataset,
            remove_columns=eval_dataset.column_names,
            num_proc=os.cpu_count(),
            fn_kwargs={"processor": processor}
        ).filter(
            lambda x: x["input_features"] is not None
        )
        print(f"필터링 후 검증 데이터셋 크기: {len(eval_dataset)}")
        print("검증 데이터 전처리 완료.")

    # 4. 트레이너 설정 및 훈련/평가 실행
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
    metric = evaluate.load("wer")
    
    training_args = Seq2SeqTrainingArguments(
        output_dir="./whisper-finetuned-finance",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=4000 if run_training else 0,
        evaluation_strategy="steps" if run_evaluation else "no",
        eval_steps=500,
        save_steps=1000,
        fp16=True,
        per_device_eval_batch_size=8,
        predict_with_generate=True,
        generation_max_length=225,
        logging_steps=100,
        report_to=["tensorboard"],
        push_to_hub=False,
        do_train=run_training,
        do_eval=run_evaluation,
        remove_unused_columns=False,
    )
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        compute_metrics=lambda pred: compute_metrics(pred, processor, metric),
        tokenizer=processor.feature_extractor,
    )

    if run_training:
        print("\n학습을 시작합니다...")
        trainer.train()
        print("학습 완료.")

    if run_evaluation:
        print("\n최종 모델을 평가합니다...")
        eval_results = trainer.evaluate(eval_dataset=eval_dataset)
        print(f"최종 WER: {eval_results['eval_wer']:.2f}%")
        
    if run_training:
        final_model_path = "./whisper-finetuned-finance/final_model"
        trainer.save_model(final_model_path)
        processor.save_pretrained(final_model_path)
        print(f"최종 모델이 '{final_model_path}'에 저장되었습니다.")

if __name__ == "__main__":
    main()

사용 장치: cpu

--- 검증 데이터 로드 시작 ---
검증 데이터 오디오 경로: C:/Users/Admin/Desktop/github/voice-recognition-using-whisper/data/news_scripts_and_speech_data/open_data\Validation\01_SourceData\VS
검증 데이터 JSON 경로: C:/Users/Admin/Desktop/github/voice-recognition-using-whisper/data/news_scripts_and_speech_data/open_data\Validation\02_LabeledData\VL
총 1606개의 Validation용 경제 도메인 데이터를 찾았습니다.
                                                path  \
0  C:/Users/Admin/Desktop/github/voice-recognitio...   
1  C:/Users/Admin/Desktop/github/voice-recognitio...   
2  C:/Users/Admin/Desktop/github/voice-recognitio...   
3  C:/Users/Admin/Desktop/github/voice-recognitio...   
4  C:/Users/Admin/Desktop/github/voice-recognitio...   

                                       transcription  
0  대부분 온라인 쇼핑몰에서 가공, 신선식품이나 일용 잡화를 판매할 때 단위 가격을 표...  
1  이에 따라 소비자의 합리적인 선택을 위해 온라인 쇼핑몰에서도 오프라인 매장처럼 단위...  
2  한국소비자원이 대형 마트 쇼핑몰 (3)/(세) 곳과 오픈 마켓 (8)/(여덟) 곳 ...  
3  대형 마트 등 오프라인 매장은 판매 가격만으로는 가격 비교가 어려운 (84)/(여든...  
4  소비자원이 쇼핑몰별로 

Map:   0%|          | 0/1606 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1606 [00:00<?, ? examples/s]

필터링 후 검증 데이터셋 크기: 1606
검증 데이터 전처리 완료.

최종 모델을 평가합니다...


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


최종 WER: 42.24%
